每 1 分鐘抓取需量資料與天氣資料寫入資料庫中
============

In [1]:
# 匯入需要的模組 [B]
%matplotlib inline
import requests
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import numpy as np # Use numpy to convert to arrays
import pandas as pd # 引用套件並縮寫為 pd  
import seaborn as sns
import pymysql
# 匯入需要的模組 [E]

In [5]:
# 參數設定 [B]
now = datetime.now()
now_hourly_string = now.strftime("%Y-%m-%d %H:%M:00")
back_hour_string = (datetime.strptime(now_hourly_string,  "%Y-%m-%d %H:%M:%S") + timedelta(minutes=-1)).strftime("%Y-%m-%d %H:%M:00")
now_hourly = datetime.strptime(now_hourly_string,  "%Y-%m-%d %H:%M:%S")
# 參數設定 [E]

In [13]:
#抓取天氣資料 [B]
try: 
    request_weather_data = requests.get('https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-61B968D9-68B1-44CD-87D6-3B39404E4A3F&locationName=%E8%87%BA%E5%8D%97')
    weather_data_result = request_weather_data.json()
    weather_data_list = weather_data_result['records']['location'][0]
    observe_time = datetime.strptime(weather_data_list['time']['obsTime'], "%Y-%m-%d %H:%M:%S")
    weather_keys_list = []
    weather_value_list = []
    weather_dataframe = []
    # if (observe_time >= now_hourly): # 代表是這個時段的天氣資料
    for i in weather_data_list['weatherElement']:
        weather_keys_list.append(i['elementName'])
        weather_value_list.append(i['elementValue'])
    weather_dataframe = pd.DataFrame(data=[weather_value_list], columns=weather_keys_list)
except Exception as e:
    print(e)
#抓取天氣資料 [E]

In [14]:
weather_data_result

{'success': 'true',
 'result': {'resource_id': 'O-A0003-001',
  'fields': [{'id': 'lat', 'type': 'Double'},
   {'id': 'lon', 'type': 'Double'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'stationId', 'type': 'String'},
   {'id': 'obsTime', 'type': 'Timestamp'},
   {'id': 'elementName', 'type': 'String'},
   {'id': 'elementValue', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'}]},
 'records': {'location': [{'lat': '22.994974',
    'lon': '120.196676',
    'locationName': '臺南',
    'stationId': '467410',
    'time': {'obsTime': '2022-11-29 09:20:00'},
    'weatherElement': [{'elementName': 'ELEV', 'elementValue': '40.8'},
     {'elementName': 'WDIR', 'elementValue': '100'},
     {'elementName': 'WDSD', 'elementValue': '0.9'},
     {'elementName': 'TEMP', 'elementValue': '27.9'},
     {'elementName': 'HUMD', 'elementValue': '0.75'},
     {'elementName': 'PRES', 'elementValue': '1012.3'},
     {'elementName': '24R',

In [15]:
weather_dataframe


,ELEV,WDIR,WDSD,TEMP,HUMD,PRES,24R,H_FX,H_XD,H_FXT,...,H_10D,H_F10T,H_UVI,D_TX,D_TXT,D_TN,D_TNT,D_TS,VIS,Weather
0,40.8,100,0.9,27.9,0.75,1012.3,0.0,2.6,80,0821,...,100,0821,3.20,28.2,0908,24.9,0626,2.5,21-30,晴


In [16]:
weather_dataframe['TEMP']

0    27.9
Name: TEMP, dtype: object

In [18]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = 'ESmeter' )  
    cur  =  conn.cursor() 
    select_sql = '''SELECT * FROM `demand` 
                        WHERE `datetime` LIKE \'%:%:00\'
                                AND `model` = \'DEM600-4\'  
                                AND `datetime` >= '{}'
                                AND `datetime` < '{}'
                        ORDER BY `datetime` '''.format(back_hour_string, now_hourly_string)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    if len(results_values_list) > 0:
        result_key_list = [i[0] for i in cur.description]
        demand_dataframe = pd.DataFrame(results_values_list)
        demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [19]:
demand_dataframe

,id,address,channel,model,datetime,demand_min,demand_quarter,Total_value
0,16308900,15,15,DEM600-4,2022-11-29 09:18:00,126.0,139.2,133.0


In [8]:
#儲存需量與天氣資料[B]
if len(results_values_list) > 0 :
    conn2  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = 'ncku_demand_minutes' )
    cur2  =  conn2.cursor()
    insert_demand_weather_sql = "INSERT INTO `demand_with_weather_data`(`demand_min`, `demand_quarter`, `Total_value`, `Temperature`, `humd`, `pres`, `w_DSD`, `w_DIR`, `h_FX`, `24R`, `h_UVI`, `T_Min`, `T_Max`, `day_of_year`, `data_week`, `data_date`, `data_hour`, `data_minute`, `data_weekday`, `minutes_of_the_day`, `datetime`) VALUES"
    innser_sql = ''
    for index, demand_info in demand_dataframe.iterrows():
    #     this_data_datetime = datetime.strptime(demand_info['datetime'], "%Y-%m-%d %H:%M:%S")
        this_data_datetime = demand_info['datetime']
        data_day_of_year = this_data_datetime.timetuple().tm_yday # 一天的第幾天;
        data_week = this_data_datetime.isocalendar()[1] # 一年當中的第幾周
        data_date = this_data_datetime.date().strftime("%Y-%m-%d")
        data_hour = this_data_datetime.hour
        data_minute = this_data_datetime.minute
        data_weekday = this_data_datetime.isoweekday()
        minutes_of_the_day = (data_hour*60 + data_minute)
        rain_24h = 0 if float(weather_dataframe['24R'][0]) < 0 else weather_dataframe['24R'][0]
        T_min = weather_dataframe['D_TN'][0] if float(weather_dataframe['D_TN'][0]) > 0 else 0
        innser_sql += ', ' if len(innser_sql) > 0 else ''
        innser_sql += " ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
        demand_info['demand_min'], demand_info['demand_quarter'], demand_info['Total_value'], weather_dataframe['TEMP'][0], weather_dataframe['HUMD'][0], weather_dataframe['PRES'][0], weather_dataframe['WDSD'][0], weather_dataframe['WDIR'][0], weather_dataframe['H_FX'][0], weather_dataframe['24R'][0], weather_dataframe['H_UVI'][0], T_min, weather_dataframe['D_TX'][0], data_day_of_year, data_week, data_date, data_hour, data_minute, data_weekday, minutes_of_the_day, demand_info['datetime'])
    insert_demand_weather_sql += innser_sql
    cur2.execute(insert_demand_weather_sql)
    conn2.commit()

#儲存需量與天氣資料[E]

In [9]:
# 儲存天氣資料 [B]
conn2  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = 'ncku_demand' )
cur2  =  conn2.cursor()
rain_24h = 0 if float(weather_dataframe['24R'][0]) < 0 else weather_dataframe['24R'][0]
insert_weather_sql = "INSERT INTO `weather_data_cwb`(`ELEV`, `WDIR`, `WDSD`, `TEMP`, `HUMD`, `PRES`, `24R`, `H_FX`, `H_XD`, `H_FXT`, `H_F10`, `H_10D`, `H_F10T`, `H_UVI`, `D_TX`, `D_TXT`, `D_TN`, `D_TNT`, `D_TS`, `VIS`, `Weather`, `observe_time`) VALUES "
insert_weather_sql += "('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
    weather_dataframe['ELEV'][0], weather_dataframe['WDIR'][0], weather_dataframe['WDSD'][0], weather_dataframe['TEMP'][0], weather_dataframe['HUMD'][0], weather_dataframe['PRES'][0], rain_24h,  weather_dataframe['H_FX'][0],  weather_dataframe['H_XD'][0],  weather_dataframe['H_FXT'][0],  weather_dataframe['H_F10'][0], 
    weather_dataframe['H_10D'][0],  weather_dataframe['H_UVI'][0],  weather_dataframe['H_F10'][0],  weather_dataframe['D_TX'][0],  weather_dataframe['D_TXT'][0],  weather_dataframe['D_TN'][0],  weather_dataframe['D_TNT'][0],  weather_dataframe['D_TS'][0],  weather_dataframe['VIS'][0],  weather_dataframe['Weather'][0], observe_time
)
cur2.execute(insert_weather_sql)
conn2.commit()
# 儲存天氣資料 [E]

In [17]:
cur2.close () 
conn2.close()